In [1]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 133 kB 5.2 MB/s 


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
tf.__version__

'2.8.2'

In [4]:
fashion_mnist=keras.datasets.fashion_mnist

In [5]:
(train_images,train_lables),(test_images,test_labels)=fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [6]:
train_images=train_images/255.0
test_images=test_images/255.0


In [8]:
train_images[0].shape

(28, 28)

In [9]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [10]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [11]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [13]:
tuner_search=RandomSearch(build_model,objective='val_accuracy',
                          max_trials=5,directory='output',
                          project_name='fashion_mnist')

In [14]:
tuner_search.search(train_images,train_lables,epochs=3,validation_split=0.1)


Trial 5 Complete [00h 00m 41s]
val_accuracy: 0.8464999794960022

Best val_accuracy So Far: 0.9068333506584167
Total elapsed time: 00h 03m 20s
INFO:tensorflow:Oracle triggered exit


In [15]:
model=tuner_search.get_best_models(num_models=1)[0]

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 48)        1248      
                                                                 
 conv2d_1 (Conv2D)           (None, 20, 20, 48)        57648     
                                                                 
 flatten (Flatten)           (None, 19200)             0         
                                                                 
 dense (Dense)               (None, 96)                1843296   
                                                                 
 dense_1 (Dense)             (None, 10)                970       
                                                                 
Total params: 1,903,162
Trainable params: 1,903,162
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.fit(train_images,train_lables,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 8s 4ms/step - loss: 0.1644 - accuracy: 0.9386 - val_loss: 0.2557 - val_accuracy: 0.9105
Epoch 5/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1252 - accuracy: 0.9529 - val_loss: 0.3018 - val_accuracy: 0.9117
Epoch 6/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0963 - accuracy: 0.9643 - val_loss: 0.3272 - val_accuracy: 0.9082
Epoch 7/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0746 - accuracy: 0.9732 - val_loss: 0.3626 - val_accuracy: 0.9112
Epoch 8/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0573 - accuracy: 0.9790 - val_loss: 0.4133 - val_accuracy: 0.9082
Epoch 9/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0465 - accuracy: 0.9824 - val_loss: 0.4013 - val_accuracy: 0.9128
Epoch 10/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0401 - accuracy: 0.9855 - val_loss: 0.5006 - val_accuracy

In [24]:
from tensorflow.keras.models import load_model
model.save('model.h5')

In [25]:
y_pred=model.predict(test_images)

In [26]:
y_pred

array([[7.8062008e-26, 4.4810521e-17, 8.6581267e-27, ..., 3.2393608e-13,
        1.0678944e-23, 1.0000000e+00],
       [9.4719170e-11, 6.9797637e-21, 9.9999821e-01, ..., 2.0351623e-18,
        1.5365727e-18, 8.1415479e-13],
       [1.9589315e-27, 1.0000000e+00, 2.5205512e-28, ..., 1.4246940e-32,
        5.5793468e-20, 4.0376628e-34],
       ...,
       [4.1181500e-25, 3.5978080e-34, 2.6851826e-27, ..., 5.9565759e-31,
        1.0000000e+00, 5.7216562e-31],
       [1.6085296e-27, 1.0000000e+00, 0.0000000e+00, ..., 2.8518670e-37,
        5.5029962e-28, 6.6342151e-34],
       [2.6088697e-15, 1.0606856e-15, 1.3457617e-15, ..., 2.4538999e-06,
        9.8745786e-12, 4.3610862e-10]], dtype=float32)